In [4]:
from deepeval.synthesizer import Synthesizer
from deepeval.dataset import Golden
from deepeval.synthesizer.config import StylingConfig
from repositories.faq import  get_all as get_all_faqs

faqs = get_all_faqs()
context_format = "Thông tin về câu hỏi và câu trả lời thường gặp của khách hàng tại FPT Shop:\nCâu hỏi: {question}\nCâu trả lời: {answer}\n"
contexts = [context_format.format(question=faq.question, answer=faq.answer) for faq in faqs]

styling_config = StylingConfig(
    input_format="Các câu hỏi bằng tiếng Việt liên quan đến sản phẩm, chính sách bảo hành, thanh toán, giao hàng, và dịch vụ tại FPT Shop.",
    expected_output_format="Câu trả lời tư vấn rõ ràng, chính xác, thể hiện sự chuyên nghiệp và thân thiện như một nhân viên bán hàng của FPT Shop.",
    task="Trả lời các câu hỏi thường gặp của khách hàng về sản phẩm và dịch vụ của FPT Shop nhằm hỗ trợ bán hàng và chăm sóc khách hàng.",
    scenario="Khách hàng tiềm năng hoặc khách hàng hiện tại đang cần được giải đáp nhanh chóng về thông tin mua sắm, chính sách và dịch vụ tại FPT Shop.",
)



In [5]:
synthesizer = Synthesizer(
    model="gpt-4o-mini",
    max_concurrent=10,
    cost_tracking=True,
    styling_config=styling_config,
)
goldens = synthesizer.generate_goldens_from_contexts(
    contexts=[[context] for context in contexts],
)

✨ Generating up to 68 goldens using DeepEval (using gpt-4o-mini, method=default): 100%|██████████| 68/68 [01:26<00:00,  1.27s/it]


💰 API cost: 0.037015

💰 API cost: 0.037015

In [1]:
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import  gen_answer

def get_actual_answer(input: str) -> str:
    user = create_user(CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user))
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(input)}],
        user_id=user.id,
    )
    

2025-05-03 10:26:22 - Loaded .env file
2025-05-03 10:26:26 - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
2025-05-03 10:26:27 - <<< {"data":{"viewer":{"username":"phatnguyen-041203","defaultEntity":{"name":"tlcn"}}}}
Logged in as Weights & Biases user: phatnguyen-041203.
View Weave data at https://wandb.ai/tlcn/CHATBOT-TLCN/weave
2025-05-03 10:26:28 - file_cache is only supported with oauth2client<4.0.0


In [ ]:
import weave
from service.faq import  search as search_faq
from deepeval.test_case import LLMTestCase
from deepeval.metrics import  AnswerRelevancyMetric, FaithfulnessMetric, ContextualPrecisionMetric, ContextualRecallMetric
from deepeval import evaluate
from service.wandb import *
from weave.flow.eval import Evaluation
import asyncio

faq_dataset = weave.ref("20250502_210525").get()


@weave.op(name="get_faq_test_case")
def get_test_case(input: str, expected_output: str, context: list[str]) -> LLMTestCase:
    faqs = search_faq(input)
    actual_answer = get_actual_answer(input)
    test_case = LLMTestCase(
        input=str(input),
        expected_output=expected_output,
        actual_output=actual_answer,
        context=context,
        retrieval_context=[context_format.format(question=faq.question, answer=faq.answer) for faq in faqs],
    )
    return test_case


@weave.op(name="evaluate_faq")
def evaluate_faq(input, model_output: LLMTestCase) -> dict:
    metrics = [
        ContextualPrecisionMetric(
            model="gpt-4o-mini", include_reason=False, async_mode=False
        ),
        ContextualRecallMetric(
            model="gpt-4o-mini", include_reason=False, async_mode=False
        ),
        AnswerRelevancyMetric(
            model="gpt-4o-mini", include_reason=False, async_mode=False
        ),
        FaithfulnessMetric(
            model="gpt-4o-mini", include_reason=False, async_mode=False
        ),
    ]

    results = evaluate(
        test_cases=[model_output],
        metrics=metrics,
    )
    metrics_results = results.test_results[0].metrics_data

    # Convert the results to a dictionary
    results_dict = {
        metrics_result.name: metrics_result.score
        for metrics_result in metrics_results or []
    }

    return results_dict

context_format = "Thông tin về câu hỏi và câu trả lời thường gặp của khách hàng tại FPT Shop:\nCâu hỏi: {question}\nCâu trả lời: {answer}\n"
evaluation = Evaluation(name="FAQ Evaluation", dataset=faq_dataset, scorers=[evaluate_faq], evaluation_name="faq_evaluation")
await evaluation.evaluate(get_test_case)

2025-05-03 10:26:31 - Using 'model_output' key for compatibility with older scorers. Please update scorers to use 'output' parameter.
🍩 https://wandb.ai/tlcn/CHATBOT-TLCN/r/call/0196942e-67b0-75f2-933e-149a68f97fea
2025-05-03 10:26:32 - Connection pool is full, discarding connection: trace.wandb.ai. Connection pool size: 10
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-03 10:26:33 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
20

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o-mini, strict=False, async_mode=True)...

...

2025-05-03 10:35:13 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:14 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:16 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:16 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:17 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:35:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:09,  9.07s/test case]



Metrics Summary

  - ✅ Contextual Precision (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)
  - ✅ Answer Relevancy (score: 0.8333333333333334, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: None, error: None)

For test case:

  - input: Làm thế nào để điều chỉnh hóa đơn sai thông tin tại FPT Shop? Tôi cần gọi số nào và cần cung cấp thông tin gì cho điện thoại viên?
  - actual output: Để điều chỉnh hóa đơn sai thông tin tại FPT Shop, Quý khách vui lòng gọi đến tổng đài 1800.6616 để gặp điện thoại viên tư vấn hỗ trợ. Khi gọi, Quý khách cần cung cấp các thông tin sau:

1. Số điện thoại mà Quý khách đã sử dụng để mua hàng.
2. Thông tin chi tiết về hóa đơn cần điều chỉnh

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

2025-05-03 10:39:14 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-03 10:39:15 - OpenAI Error: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=16384, prompt_tokens=1055, total_tokens=17439, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)) Retrying: 2 time(s)...
2025-05-03 10:43:47 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
